In [17]:
#----------------- LOAD EXTERNAL LIBRARIES AND MODULES-------------------------------------------------------------------
require 'bio'
require 'net/http'


false

In [ ]:
#------------------------------ INPUT FILES NAMES AS ARGUMENTS FROM  COMMAND LINE ---------------------------------------

# Check for one common error: not specifying the correct number of files needed for the program to run
if ARGV.length != 1
    abort "Incorrect number of files passed. Two files names must be specified: input list of genes and name for final_report"
end
  
# Check for second common error: incorrect usage, files in incorrect order or wrong name passed.
#if ARGV[0] == "ArabidopsisSubNetwork_GeneList.txt"
if ARGV[0] == 'soloungen.txt'
        gene_file = ARGV[0]
else 
    abort "Incorrect order of files passed. Usage: ruby main.rb ArabidopsisSubNetwork_GeneList.txt"
end


In [18]:
# ------------ PUBLIC INSTANCE METHODS ---------------------------------------

# Get response from URL
def fetch(uri_str) 
    address = URI(uri_str)  
    response = Net::HTTP.get_response(address)
    case response
      when Net::HTTPSuccess then
        return response.body
      else
        raise Exception, "Something went wrong... the call to #{uri_str} failed; type #{response.class}"
        response = false
        return response
    end
end

# Get response with NCBI E-Utils, given a database, file format and gene locus
def ncbi_fetch(database, file_format, id)
    url = "http://www.ebi.ac.uk/Tools/dbfetch/dbfetch?db=#{database}&format=#{file_format}&id=#{id}"
    #puts url
    res = fetch(url)
    return res
end


# Transform locus names file into string of coma separated locus names, while checking A. thaliana format
def read_from_file(filename)
    gene_list = []
    gene_file = File.open(filename, 'r')
    gene_file.readlines.each do |line|  
        locus_name=line.chomp
        if locus_name !~ /A[Tt]\d[Gg]\d\d\d\d\d/ # Check for correct format of gene locus name
            abort "Locus name #{locus_name} does not meet the correct format. Please define locus names as AT0g00000"
        end
        gene_list << locus_name 
    end
    gene_file.close
    gene_ids = gene_list.join(",")
    return gene_ids
end

# Gets coordinates from string
def get_coordinates(coordinates_string)
    position = coordinates_string.match(/\d+\.\.\d+/)[0]
    start_coordinate = position.split('..')[0].to_i
    end_coordinate = position.split('..')[1].to_i
    return start_coordinate, end_coordinate
end




:get_coordinates

In [ ]:

#-------------------------------------------MAIN CODE ------------------------------------------------------------------
# BEFORE EVERYTHING, SEARCHING EMBL FILE (Task 1)

# 1:  Using BioRuby, examine the sequences of the ~167 Arabidopsis genes from the last assignment by retrieving them from whatever database you wish #
# ArabidopsisSubNetwork file -> each gene locus (lines) -> search for embl file -> add to 'AT_sequences.embl'

puts "Processing #{gene_file} file, this might take a while..."

# Create string with gene IDs from file
gene_ids = read_from_file(gene_file)

        # Only one search with list of IDs
response_body = ncbi_fetch(database = 'ensemblgenomesgene',file_format = 'embl', id = gene_ids)

output_file = File.open('AT_sequences.embl', 'w')
output_file.write(response_body)
output_file.close

In [19]:
class EmblProcessor
    
    attr_accessor :entries

    # the emblprocessor is the embl flat file so entries are instances of the class entry
    def initialize(filename)
        @entries = scan_repetitive_features(filename)
    end

    def scan_repetitive_features(filename)
        embl_file = Bio::FlatFile.auto(filename)    # crea el objeto embl flat file
        entries = []

        embl_file.each_entry do |entry| # como antes recorre las entrys
            entries << entry   # add each entry instance to the array with all the entrys
        end
        
        return entries
    end  
end


:scan_repetitive_features

In [7]:
class EmblEntry

    attr_accessor :entry_sequence, :chromosomal_coordinates, :source

    def initialize
        @sequence_entry = entry.to_biosequence
        @chromosomal_coordinates = [start_entry, end_entry]
        @source = ""
    end
end



:initialize

In [ ]:
class EmblProcessor
  attr_accessor :entries

  def initialize(filename)
    @entries = scan_repetitive_features(filename)
  end

  def scan_repetitive_features(filename)
    embl_file = Bio::FlatFile.auto(filename)
    entries = []

    embl_file.each_entry do |entry|
      start_entry, end_entry = get_coordinates(entry.definition)
      embl_entry = EmblEntry.new(entry, start_entry, end_entry)
      embl_entry.process_features
      entries << embl_entry
    end

    entries
  end

  def load_to_gff(coordinates_to_use)
    gff = Bio::GFF::GFF3.new
    count = 1

    @entries.each do |embl_entry|
      embl_entry.process_cttctt_repeats(coordinates_to_use)

      embl_entry.entry_sequence.features.each do |feature|
        next unless feature.feature == 'cttctt_repeat'

        qual = feature.assoc
        attributes = [{ 'ID' => "repeat_region_#{count}", 'Name' => 'CTTCTT_motif' }]

        seqid = embl_entry.seq_id

        gff.records << Bio::GFF::GFF3::Record.new(
          seqid,            # seqID
          qual['source'],   # source
          qual['SO_Name'],  # feature type
          qual['start'],    # start
          qual['end'],      # end
          nil,              # score
          qual['strand'],   # strand
          nil,              # phase
          attributes[0]     # attributes
        )
        count += 1
      end
    end

    gff
  end
end

class EmblEntry
  attr_accessor :entry_sequence, :chromosomal_coordinates, :source

  def initialize(entry, start_entry, end_entry)
    @entry_sequence = entry.to_biosequence
    @chromosomal_coordinates = [start_entry, end_entry]
    @source = ""
  end

  def add_chromosomal_coordinates
    f1 = Bio::Feature.new('chromosomal_coordinates', @entry_sequence.accession)
    f1.append(Bio::Feature::Qualifier.new('start', @chromosomal_coordinates[0]))
    f1.append(Bio::Feature::Qualifier.new('end', @chromosomal_coordinates[1]))
    @entry_sequence.features << f1
  end

  def add_gene_from_list(gene_name)
    return if @entry_sequence.features.any? { |feature| feature.feature == 'gene_from_list' }
    f1 = Bio::Feature.new('gene_from_list', gene_name)
    @entry_sequence.features << f1
  end

  def process_features
    add_chromosomal_coordinates

    @entry_sequence.features.each do |feature|
      featuretype = feature.feature
      position = feature.position
      qual = feature.assoc

      if featuretype == 'source'
        @source = qual['db_xref']
      end

      if featuretype == 'gene' && position.match(/^(complement\()?(\d+\.\.\d+)(\))?$/)
        add_gene_from_list(qual['gene'])
      end
    end
  end

  def process_cttctt_repeats(coordinates_to_use)
    regex_positive = Regexp.new(search_positive.to_re)
    regex_complementary = Regexp.new(search_complementary.to_re)
    
    @entry_sequence.features.each do |feature|
      next unless feature.feature == 'exon' && feature.position.match?(/^(complement\()?(\d+\.\.\d+)(\))?$/)

      start_exon, end_exon = get_coordinates(feature.position)
      exon_sequence = @entry_sequence.subseq(start_exon, end_exon)

      next if exon_sequence.nil? || exon_sequence.empty?

      if feature.position.include?('complement')
        regex = regex_complementary
        strand = '-'
        coordinates_format = "complement(%s)"
      else
        regex = regex_positive
        strand = '+'
        coordinates_format = "%s"
      end

      matches_motifs = find_motifs(exon_sequence, regex)
      next if matches_motifs.empty? || matches_motifs.nil?

      matches_motifs.each do |match|
        start_match = match[:start]
        end_match = match[:end]

        start_motif = start_exon + start_match
        end_motif = start_exon + end_match - 1

        if coordinates_to_use == 'chromosomal coordinates'
          start_motif, end_motif = convert_to_chromosomal_coordinates(start_motif, end_motif)
        end

        coordinates = coordinates_format % "#{start_motif}..#{end_motif}"

        add_cttctt_repeat(coordinates, start_motif, end_motif, strand, @source)
      end
    end
  end

  def add_cttctt_repeat(coordinates, start_motif, end_motif, strand, source)
    return if @entry_sequence.features.any? { |f| f.feature == 'cttctt_repeat' && f.position == coordinates }

    f1 = Bio::Feature.new('cttctt_repeat', coordinates)
    f1.append(Bio::Feature::Qualifier.new('start', start_motif))
    f1.append(Bio::Feature::Qualifier.new('end', end_motif))
    f1.append(Bio::Feature::Qualifier.new('strand', strand))
    f1.append(Bio::Feature::Qualifier.new('source', source))
    f1.append(Bio::Feature::Qualifier.new('SO_Name', 'repeat_region'))
    @entry_sequence.features << f1
  end

  def convert_to_chromosomal_coordinates(start_motif, end_motif)
    chr_coordinates = @entry_sequence.features.find { |f| f.feature == 'chromosomal_coordinates' }
    qual_chr = chr_coordinates.assoc
    start_chr = qual_chr['start']
    end_chr = qual_chr['end']

    length_motif = end_motif - start_motif
    start_motif = start_motif + start_chr - 1
    end_motif = start_motif + length_motif - 1

    [start_motif, end_motif]
  end
end

def write_gff(gff, coordinates_to_use)
  file = coordinates_to_use == 'chromosomal coordinates' ? 'AT_repeats_chromosomal.gff' : 'AT_repeats_sequence.gff'

  File.open(file, 'w') do |file|
    file.puts gff.to_s
  end
end

#-------------------------------------------MAIN CODE ------------------------------------------------------------------

# BEFORE EVERYTHING, SEARCHING EMBL FILE (Task 1)
# ...
# FIRST PART WITH SEQUENCE COORDINATES (Tasks 2, 3, 4a, 4b)
processor = EmblProcessor.new('AT_sequences.embl')
gff = processor.load_to_gff('sequences coordinates')
write_gff(gff, 'sequences coordinates')


In [ ]:
require 'bio'

# Replace 'your_embl_file.embl' with the path to your EMBL file
embl_file_path = 'your_embl_file.embl'

# Create a new GFF3 file (replace 'output.gff3' with your desired output file name)
gff3_file_path = 'output.gff3'
gff3_file = File.open(gff3_file_path, 'w')

# Read the EMBL file
embl_entries = Bio::FlatFile.auto(embl_file_path)

# Iterate through each entry in the EMBL file
embl_entries.each do |entry|
  # Extract information from the EMBL entry to create GFF3 entries
  entry.features.each do |feature|
    # Customize this section based on your EMBL file structure and desired GFF3 format
    seqid   = entry.definition
    source  = 'YourSource'  # Provide a suitable source name
    type    = feature.feature
    start   = feature.position.from
    end_pos = feature.position.to
    score   = '.'  # Replace with the appropriate score if available
    strand  = feature.position.strand
    phase   = '.'  # Replace with the appropriate phase if available

    # Build the GFF3 line
    gff3_line = "#{seqid}\t#{source}\t#{type}\t#{start}\t#{end_pos}\t#{score}\t#{strand}\t#{phase}\t"

    # Add attributes based on the EMBL feature qualifiers
    attributes = feature.qualifiers.map { |qualifier| "#{qualifier.qualifier}=#{qualifier.value}" }
    gff3_line += attributes.join(';')

    # Write the GFF3 line to the output file
    gff3_file.puts(gff3_line)
  end
end

# Close the GFF3 file
gff3_file.close